# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski

### Instructions
- Sign up for RunPod here: https://runpod.io/?ref=n8yfwyum
    - Note: That's my personal referral link. Please don't use it if we are mortal enemies.

- Click *Deploy* on either `SECURE CLOUD` or `COMMUNITY CLOUD`

- Follow the rest of the instructions in this video: https://www.youtube.com/watch?v=7m__xadX0z0#t=5m33.1s

Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [2]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1


     |████████████████████████████████| 79 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 117 kB 37.6 MB/s eta 0:00:01
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=69fee6e9ce11cbb08631df29d940b276e3c4742d0ba18d5cc3691699e47b3175
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime
     |████████████████████████████████| 585 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 85.9 MB/s eta 0:00:01
     |████████████████████████████████| 529 kB 70.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.9 MB 80.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 99.4 MB/s eta 0:00:01
     |████████████████████████████████| 948 kB 74.7 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 148 kB 93.

In [2]:
# Hugging Face Login
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# Download the 1.4 sd model
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="CompVis/stable-diffusion-v-1-4-original",
 filename="sd-v1-4.ckpt",
 use_auth_token=True
)

# Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

✅ model.ckpt successfully downloaded


# Regularization Images (Skip this section if you are uploading your own or using the provided images)

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [ ]:
# GENERATE 200 images - Optional
self_generated_files_prompt = "person" #@param {type:"string"}
self_generated_files_count = 200 #@param {type:"integer"}

!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter {self_generated_files_count} \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt {self_generated_files_prompt}

dataset=self_generated_files_prompt

!mkdir -p regularization_images/{dataset}
!mv outputs/txt2img-samples/*.png regularization_images/{dataset}

In [ ]:
# Zip up the files for downloading and reuse.
# Download this file locally so you can reuse during another training on this dataset
!apt-get install -y zip
!zip -r regularization_images.zip regularization_images/{dataset}

# Download pre-generated regularization images
We've created the following image sets

`man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
`man_unsplash` - pictures from various photographers
`person_ddim`
`woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
`person_ddim` is recommended

In [1]:
#Download Regularization Images

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "blonde_woman"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

In [ ]:
#@markdown Add here the URLs to the images of the subject you are adding
urls = [
 "https://i.imgur.com/test1.png",
 "https://i.imgur.com/test2.png",
 "https://i.imgur.com/test3.png",
 "https://i.imgur.com/test4.png",
 "https://i.imgur.com/test5.png",
 # You can add additional images here -- about 20-30 images in different
]

In [ ]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid

def download_image(url):
 try:
  response = requests.get(url)
 except:
  return None
 return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./training_images"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

# Rename all the icons

In [8]:
import os
import random
files = os.listdir('res/regularization_images/')
# pick 40 random images
random.seed(1234)
files = random.choices(files, k=40)
assert(len(files) == 40)
counter = 0
# and move them to the training directory
for file in files:
    os.rename(os.path.join('res/regularization_images', file),
              os.path.join('res/fontawesome-png', 'randomicontest' + str(counter).zfill(3) + '.png'))
    counter += 1

In [ ]:
# img_dir = 'res/fontawesome-png/'
# img_dir_out = 'res/fontawesome-png-ready/'
# import os
# os.mkdir(img_dir_out)
# files = os.listdir(img_dir)
# # rename
# counter = 0
# for file in files:
#     os.rename(os.path.join(img_dir, file),
#               os.path.join(img_dir_out, 'randomicontest' + str(counter).zfill(3) + '.png'))
#     counter += 1     

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [11]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "randomicontest"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 1500

# Match class_word to the category of the regularization images you chose above.
class_word = "black SVG icon" # typical uses are "man", "person", "woman"

# This is the unique token you are incorporating into the stable diffusion model.
token = "randomicontest"


reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/res/regularization_images/"

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/res/fontawesome-png" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

Global seed set to 23
Running on GPUs 0,
Loading model from model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.3.layer_norm1.bias', 'vision_model.encoder.layers.22.layer_norm1.weight', 'vision_model.encoder.layers.3.self_attn.q_proj.weight', 'vision_model.encoder.layers.18.layer_norm1.weight', 'vision_model.encoder.layers.13.layer_norm1.bias', 'vision_model.encoder.layers.22.mlp.fc1.weight', 'vision_model.encoder.layers.9.mlp.fc1.bias', 'vision_model.encoder.layers.12.self_attn.k_proj.weight', 'vision_model.encoder.layers.5.self_attn.k_proj.weight', 'vision_model.encoder.layers.4.self_attn.q_proj.bias', 'vision_model.encoder.lay

## Copy and name the checkpoint file

In [10]:
# Copy the checkpoint into our `trained_models` folder

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

Download your trained model file from trained_models/2022-11-08T02-06-00_randomicontest_1_training_images_1500_max_training_steps_randomicontest_token_black_SVG_icon_class_word.ckpt and use in your favorite Stable Diffusion repo!


# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
# from https://icons8.com/icons/request-icon
prompts = ['a hyena', 'a fox', 'a playstation', 'trash', 'laundry', 
           'soup', 'a yogurt container', 'fennel', 'a keyboard',
           'instagram icon', 'the sims icon', 'galaxy app store',
           'amazon app store', 'world of tanks',
           'python', 'steam store', 'a phone', 'discord logo',
           'twitter logo', 'jupyter notebook',
           'a crocodile holding a computer',
           'jesus christ crucifix', 'pet family', 'lily flower',
           'rose flower', 'elon musk',
           'peace sign symbol', 'flexing muscle', 'flexing arm muscle',
           'fisherman boat',
           'a paintbrush', 'skiing']

for prompt in prompts:
    !python scripts/stable_txt2img.py \
     --ddim_eta 0.0 \
     --n_samples 10 \
     --n_iter 4 \
     --scale 7.0 \
     --ddim_steps 50 \
     --ckpt "/workspace/Dreambooth-Stable-Diffusion/logs/fontawesome-png2022-11-08T02-07-03_randomicontest/checkpoints/last.ckpt" \
     --prompt "black SVG icon of {prompt}"

Global seed set to 42
Loading model from /workspace/Dreambooth-Stable-Diffusion/logs/fontawesome-png2022-11-08T02-07-03_randomicontest/checkpoints/last.ckpt
Killed
Global seed set to 42
Loading model from /workspace/Dreambooth-Stable-Diffusion/logs/fontawesome-png2022-11-08T02-07-03_randomicontest/checkpoints/last.ckpt
Killed
^C
Traceback (most recent call last):
  File "scripts/stable_txt2img.py", line 2, in <module>
    import torch
  File "/opt/conda/lib/python3.7/site-packages/torch/__init__.py", line 202, in <module>
    from torch._C import *  # noqa: F403
RuntimeError: KeyboardInterrupt: 
^C


In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"